In [3]:
#using alpha_vantage because quandl gives only dattsa till March 2018

from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, date
import requests
import pandas as pd
import time
import quandl
import pandas as pd
import sys
import math
from alpha_vantage.techindicators import TechIndicators
from datapackage import Package

In [4]:
def all_time_rsi_values(stock_para):
    ts = TechIndicators(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
    data, meta_data = ts.get_rsi(symbol='AAPL')
    return data['RSI'][-1]

In [5]:
def daily_rsi_values(stock_para):
    ts = TechIndicators(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
    try:
        data, meta_data = ts.get_rsi(symbol=stock_para,interval='1min')
    except:
        return -1
    ret_val = 1
    if(data['RSI'][-1]>70 or data['RSI'][-1]<30):
        ret_val = 2
    else:
        ret_val = 1
    return ret_val

In [6]:
def get_all_data(symbol_para):
    ts = TimeSeries(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
    try:
        data, meta_data = ts.get_daily_adjusted(symbol=symbol_para, outputsize='full')
    except:
        return pd.DataFrame()
    myData = data.rename(index=str, columns={"2. high": "High", "3. low": "Low","4. close":"Close"})
    myData['20d'] = np.round(myData.Close.rolling(window =20, center = False).mean(),2)
    myData['50d'] = np.round(myData.Close.rolling(window =50, center = False).mean(),2)
    myData['200d'] = np.round(myData.Close.rolling(window =200, center = False).mean(),2)
    return myData

In [7]:
#SMA rating
def SMA_rating(data_simple):
#     start_date = date(1998,1,2)
#     yesterday = (datetime.now() - timedelta(days=1))
#     end_date = date(yesterday.year,yesterday.month,yesterday.day)
    sma_rating = 0
    index = len(data_simple.index)-1
    if data_simple['Close'][index] > data_simple['50d'][index] > data_simple['200d'][index]:
        sma_rating=1
    elif data_simple['50d'][index] > data_simple['Close'][index]> data_simple['200d'][index]:
        sma_rating=2
    elif data_simple['50d'][index] > data_simple['200d'][index] > data_simple['Close'][index]:
        sma_rating=3
    elif data_simple['200d'][index] > data_simple['50d'][index] > data_simple['Close'][index]:
        sma_rating=4
    elif data_simple['200d'][index]> data_simple['Close'][index] > data_simple['50d'][index]:
        sma_rating=5
    elif data_simple['Close'][index] > data_simple['200d'][index] > data_simple['50d'][index]:
        sma_rating=6
    elif data_simple['Close'][index] > data_simple['50d'][index] > data_simple['200d'][index]:
        sma_rating=7
    return sma_rating

In [8]:
def MACD_rating(data_simple):
        data_simple['26 ema']=data_simple.Close.ewm(span=26).mean()
        data_simple['12 ema']=data_simple.Close.ewm(span=12).mean()
        data_simple['MACD'] = (data_simple['12 ema'] - data_simple['26 ema'])
        data_simple['signal_line']=data_simple.MACD.ewm(span=9).mean()
        data_simple['hist']=(data_simple['MACD']-data_simple['signal_line'])
        
        #MACD rating
        index = len(data_simple)-1
        slope = 0
        macd_rating = 0
        
        print("Current:",data_simple['MACD'][index])
        print("Previous day",data_simple['MACD'][index-1])
        
        #Condition for the slope
        if((data_simple['MACD'][index] - data_simple['MACD'][index-1])>0):
            slope = 1
        elif((data_simple['MACD'][index] - data_simple['MACD'][index-1])<0):
            slope = -1
            
        #Condition for the MACD rating
        if data_simple['MACD'][index] > data_simple['signal_line'][index] and data_simple['signal_line'][index] > 0:
            macd_rating=1
        elif data_simple['signal_line'][index] > data_simple['MACD'][index] and data_simple['MACD'][index] > 0:
            macd_rating=2
        elif data_simple['MACD'][index] < 0 and 0 < data_simple['signal_line'][index]:
            macd_rating=3
        elif data_simple['MACD'][index] < data_simple['signal_line'][index] and data_simple['signal_line'][index]< 0:
            macd_rating=4
        elif data_simple['signal_line'][index] < data_simple['MACD'][index] and data_simple['MACD'][index] < 0:
            macd_rating=5
        elif data_simple['MACD'][index] > 0 and 0 > data_simple['signal_line'][index]:
            macd_rating=6
        return macd_rating,slope

In [9]:
def get_intra_day_data():
    ts = TimeSeries(key='ZSAE2CXUXOLE67NH', output_format='pandas')
    intra_day_data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
    intra_day_data = intra_day_data.rename(index=str, columns={"2. high": "High", "3. low": "Low","4. close":"Close"})
    intra_day_data['20m'] = np.round(intra_day_data.Close.rolling(window =20, center = False).mean(),2)
    intra_day_data['50m'] = np.round(intra_day_data.Close.rolling(window =50, center = False).mean(),2)
    intra_day_data['200m'] = np.round(intra_day_data.Close.rolling(window =200, center = False).mean(),2)
    return intra_day_data

In [10]:
#SMA rating
def SMA_rating_intra_day(data_simple):
#     start_date = date(1998,1,2)
#     yesterday = (datetime.now() - timedelta(days=1))
#     end_date = date(yesterday.year,yesterday.month,yesterday.day)
    sma_rating = 0
    sma_ratings = []
    for index,val in enumerate(data_simple['Close']):
        if data_simple['Close'][index] > data_simple['50m'][index] > data_simple['200m'][index]:
            sma_rating=1
        elif data_simple['50m'][index] > data_simple['Close'][index]> data_simple['200m'][index]:
            sma_rating=2
        elif data_simple['50m'][index] > data_simple['200m'][index] > data_simple['Close'][index]:
            sma_rating=3
        elif data_simple['200m'][index] > data_simple['50m'][index] > data_simple['Close'][index]:
            sma_rating=4
        elif data_simple['200m'][index]> data_simple['Close'][index] > data_simple['50m'][index]:
            sma_rating=5
        elif data_simple['Close'][index] > data_simple['200m'][index] > data_simple['50m'][index]:
            sma_rating=6
        elif data_simple['Close'][index] > data_simple['50m'][index] > data_simple['200m'][index]:
            sma_rating=7
        sma_ratings.append(sma_rating)
    return sma_ratings

In [11]:
def MACD_rating_intra_day(data_simple):
        data_simple['26 ema']=data_simple.Close.ewm(span=26).mean()
        data_simple['12 ema']=data_simple.Close.ewm(span=12).mean()
        data_simple['9 ema']=data_simple.Close.ewm(span=9).mean()
        data_simple['MACD'] = (data_simple['12 ema'] - data_simple['26 ema'])
        data_simple['signal_line']=data_simple.MACD.ewm(span=9).mean()
        data_simple['hist']=(data_simple['MACD']-data_simple['signal_line'])
        #MACD rating
        index = len(data_simple.index)-1
        macd_rating = 0
        macd_ratings = []
        for index,val in enumerate(data_simple['Close']):
            if data_simple['MACD'][index] > data_simple['signal_line'][index] and data_simple['signal_line'][index] > 0:
                macd_rating=1
            elif data_simple['signal_line'][index] > data_simple['MACD'][index] and data_simple['MACD'][index] > 0:
                macd_rating=2
            elif data_simple['MACD'][index] < 0 and 0 < data_simple['signal_line'][index]:
                macd_rating=3
            elif data_simple['MACD'][index] < data_simple['signal_line'][index] and data_simple['signal_line'][index]< 0:
                macd_rating=4
            elif data_simple['signal_line'][index] < data_simple['MACD'][index] and data_simple['MACD'][index] < 0:
                macd_rating=5
            elif data_simple['MACD'][index] > 0 and 0 > data_simple['signal_line'][index]:
                macd_rating=6
            macd_ratings.append(macd_rating)
        return macd_ratings

In [12]:
def daily_rsi_values(stock_para):
    ts = TechIndicators(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
    try:
        data, meta_data = ts.get_rsi(symbol=stock_para,interval='1min')
    except:
        return -1
    if(data.empty):
        return -1
    if(data['RSI'][-1]>70 or data['RSI'][-1]<30):
        ret_val = -1
    else:
        ret_val = 1
    return ret_val

In [13]:
## Global Declarations

reversal = 3 #reversal amount set to 3 as default

# (price range, box size)
#function will provide box sizes according to traditional method
box_ranges = [(.25,.0625),
              (1,.125), 
              (5,.25),
              (20,.5),
              (100,1),
              (200,2),
              (500,4),
              (1000,5),
              (25000,50),
              (sys.maxsize,500)]

#to check and update box size according to current value of stock
def updateBoxSize(price):
    for i in range (len(box_ranges)):
        if price < box_ranges[i][0]:
            return box_ranges[i][1]
    return None


#using alpha_vantage because quandl gives only data till March 2018

#if current trend is x update x column appropriately or move to o column if needed
def updateX(item, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes):
    box_size = updateBoxSize(list1[-1])
   # print(math.floor(item["High"]))
    if ( math.floor(item["High"]) >= list1[-1]+box_size ):
        list1[-1] = math.floor(item["High"])
    #   print("Updated the x value to:"+str(list1[-1]))
        numberofXBoxes += 1
    elif ( math.floor(item["Low"]) <= list1[-1]-reversal*box_size):
        list2.append(math.floor(item["Low"]))
    #   print("Updated the o value to:"+str(list2[-1]))
        current_trend = 'o'
        numberofOBoxes += 1
    return current_trend

#if current trend is o update o column appropriately or move to x column if needed
def updateO(item, list1, list2, box_size, reversal, current_trend,  numberofXBoxes, numberofOBoxes):
    box_size = updateBoxSize(list2[-1])
    if ( math.floor(item["Low"]) <= list2[-1]-box_size ):
        list2[-1] = math.floor(item["Low"])
#        print("Updated the o value to:"+str(list2[-1]))
        numberofOBoxes += 1
    elif ( math.floor(item["High"]) >= list2[-1]+reversal*box_size):
        list1.append(math.floor(item["High"]))
 #       print("Updated the x value to:"+str(list1[-1]))
        current_trend = 'x'
        numberofXBoxes += 1
    return current_trend

#create the point and figure from scratch

def pointAndFigureCreate(box_size,current_trend, list1, list2, stockhilowdata, numberofXBoxes, numberofOBoxes):
    #iterate over each date for the stock and create x or o columns
    for index,row in stockhilowdata.iloc[0:].iterrows():
        if current_trend == 'x':
        #   print("The price is:"+str(row["High"]))
            current_trend = updateX(row, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes)
        elif current_trend=='o':
            current_trend = updateO(row, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes)
          #  print("The price is:"+str(row["Low"]))
    return current_trend

#check for a continous triple top pattern
def continous_triple_top(current_trend, list1):
    if(len(list1)<3):
        return -1
    ret_val = 0
    if current_trend == 'x':
        if((list1[-1]> list1[-2]) and (list1[-2] == list1[-3])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val
        
#check for a continous triple bottom pattern
def continous_triple_bottom(current_trend, list2):
    if(len(list2)<3):
        return -1
    ret_val = 0
    if current_trend == 'o':
        if((list2[-1] < list2[-2]) and (list2[-2] == list2[-3])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

#check for a continous double top pattern
def continous_double_top(current_trend, list1):
    if(len(list1)<2):
        return -1
    ret_val = 0
    if current_trend=='x':
        if(list1[-1]> list1[-2]):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

#check for a continous double bottom pattern
def continous_double_bottom(current_trend, list2):
    if(len(list2)<2):
        return -1
    ret_val = 0
    if current_trend == 'o':
        if(list2[-1]< list2[-2]):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def continous_quadruple_top(current_trend, list1):
    if(len(list1)<4):
        return -1
    ret_val = 0
    if current_trend == 'x' and len(list1)>3:
        if((list1[-1] > list1[-2]) and (list1[-2] == list1[-3] == list1[-4])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def continous_quadruple_bottom(current_trend, list2):
    if(len(list2)<4):
        return -1
    ret_val = 0
    if current_trend == 'o' and len(list2)>3:
        if((list2[-1] < list2[-2]) and (list2[-2] == list2[-3] == list2[-4])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

import itertools
def spread_triple_top(current_trend, list1):
    ret_val = -1
    if current_trend == 'x' and len(list1)>7:
        iterprev = 0
        resistance = 0
        notstt = True
        for iter in list1[-2:-7:1]:
            if iterprev == iter:
                resistance = iter
                notstt = False
            if resistance!=0 and iter > resistance:
                notstt = True
                break
            iterprev = iter
        if(notstt == False):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def spread_triple_bottom(current_trend, list2):
    ret_val = -1
    if current_trend == 'o' and len(list2)>7:
        iterprev = 0
        support = 0
        notstb = True
        for iter in list2[-2:-7:1]:
            if iterprev == iter:
                support = iter
                notstb = False
            if support!=0 and iter<resistance:
                notstb = True
                break
            iterprev = iter
        if(notstb == False):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def PnFMain(myData):
    #list1 stores the highest value of each x column by index
    #list2 stores the lowest value of each o column by index
    list1 = []
    list2 = []

    #current_trend stores the value of the most recent trend. we start off with x as default
    current_trend = 'x'

    numberofXBoxes = 1 # to get number of x boxes in all
    numberofOBoxes = 0 # to get number of o boxes in all

    #set box size according to price on day 1
    box_size = updateBoxSize(myData["High"].iloc[0])
    #append price on day 1 to list1 as we start with x
    list1.append(math.floor(myData["High"].iloc[0]))
    current_trend = pointAndFigureCreate(box_size, current_trend, list1, list2, myData, numberofXBoxes, numberofOBoxes)
    ret_val_triple_top = continous_triple_top(current_trend,list1)
    ret_val_triple_bottom = continous_triple_bottom(current_trend,list2)
    ret_val_double_top = continous_double_top(current_trend,list1)
    ret_val_double_bottom = continous_double_bottom(current_trend,list2)
    ret_val_quadruple_top = continous_quadruple_top(current_trend,list1)
    ret_val_quadruple_bottom = continous_quadruple_bottom(current_trend,list2)
    ret_val_spread_triple_top = spread_triple_top(current_trend,list1)
    ret_val_spread_triple_bottom = spread_triple_bottom(current_trend,list2)
    
    return ret_val_double_top

In [14]:
financial_list=list()
energy_list=list()
industrial_list=list()
comm_services_list=list()
materials_list=list()
health_list=list()
consumer_list=list()
technology_list=list()
consumer_stap_list=list()

sector_etf_list=['XLF','XLE','XLI','XLC','XLB','XLV','XLY','XLK','XLP']
sector_sma_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_macd_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_macd_slope={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_pnf_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_rsi_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_etf_to_names={'XLF':financial_list,'XLE':energy_list,'XLI':industrial_list,'XLC':comm_services_list,'XLB':materials_list,'XLV':health_list,'XLY':consumer_list,'XLK':technology_list,'XLP':consumer_stap_list}


In [180]:
def get_sector_ratings(etf,sector_sma_rating,sector_macd_rating,sector_macd_slope):
    
    ts = TimeSeries(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
    data, meta_data = ts.get_daily_adjusted(symbol=etf, outputsize='full')
    myData = data.rename(index=str, columns={"2. high": "High", "3. low": "Low","4. close":"Close"})
    #print(myData.head())
    myDataSimple=myData[['Close']].copy()
    myDataSimple['20d'] = np.round(myDataSimple.Close.rolling(window =20, center = False).mean(),2)
    myDataSimple['50d'] = np.round(myDataSimple.Close.rolling(window =50, center = False).mean(),2)
    myDataSimple['200d'] = np.round(myDataSimple.Close.rolling(window =200, center = False).mean(),2)
    sma_rating_today=SMA_rating(myDataSimple)
    macd_rating_today,slope=MACD_rating(myDataSimple)
    time.sleep(12)
    return sma_rating_today,macd_rating_today,slope   


for etf in sector_etf_list:
    #print(etf)
    sector_sma_rating[etf],sector_macd_rating[etf],sector_macd_slope[etf]=get_sector_ratings(etf,sector_sma_rating,sector_macd_rating,sector_macd_slope)
    

Current: 0.2601239255465586
Previous day 0.24520698574060518
Current: 0.23928270274446106
Previous day 0.23567558665413912
Current: 0.5694725652463006
Previous day 0.6265267321591352
Current: 0.5174035868908504
Previous day 0.5123370080419605
Current: 0.7306129000482997
Previous day 0.7984575972974071
Current: 0.9454469908531138
Previous day 1.0160855303709724
Current: 1.416299788114074
Previous day 1.3451538509293641
Current: 1.1061309335997436
Previous day 1.1286157191468078
Current: 0.41101002471759784
Previous day 0.38500770526624706


In [15]:
package = Package('https://datahub.io/core/s-and-p-500-companies/datapackage.json')

In [16]:
for resource in package.resources:
    if resource.descriptor['datahub']['type'] == 'derived/csv':
        my_list=resource.read()
df = pd.DataFrame(my_list, columns=['Symbol','Name','Sector'])

In [17]:
lis1= df[1:len(df)]['Symbol']
print(lis1[1])

AOS


In [18]:
for i in range(0,len(df)):
    if df['Sector'][i]=='Industrials':
        industrial_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Health Care':
        health_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Energy':
        energy_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Financials':
        financial_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Information Technology':
        technology_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Consumer Discretionary':
        consumer_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Materials':
        materials_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Telecommunication Services':
        comm_services_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Consumer Staples':
        consumer_stap_list.append(df['Symbol'][i])

In [19]:
## Global Declarations

reversal = 3 #reversal amount set to 3 as default

# (price range, box size)
#function will provide box sizes according to traditional method
box_ranges = [(.25,.0625),
              (1,.125), 
              (5,.25),
              (20,.5),
              (100,1),
              (200,2),
              (500,4),
              (1000,5),
              (25000,50),
              (sys.maxsize,500)]

#to check and update box size according to current value of stock
def updateBoxSize(price):
    for i in range (len(box_ranges)):
        if price < box_ranges[i][0]:
            return box_ranges[i][1]
    return None


#using alpha_vantage because quandl gives only data till March 2018

#if current trend is x update x column appropriately or move to o column if needed
def updateX(item, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes):
    box_size = updateBoxSize(list1[-1])
   # print(math.floor(item["High"]))
    if ( math.floor(item["High"]) >= list1[-1]+box_size ):
        list1[-1] = math.floor(item["High"])
    #   print("Updated the x value to:"+str(list1[-1]))
        numberofXBoxes += 1
    elif ( math.floor(item["Low"]) <= list1[-1]-reversal*box_size):
        list2.append(math.floor(item["Low"]))
    #   print("Updated the o value to:"+str(list2[-1]))
        current_trend = 'o'
        numberofOBoxes += 1
    return current_trend

#if current trend is o update o column appropriately or move to x column if needed
def updateO(item, list1, list2, box_size, reversal, current_trend,  numberofXBoxes, numberofOBoxes):
    box_size = updateBoxSize(list2[-1])
    if ( math.floor(item["Low"]) <= list2[-1]-box_size ):
        list2[-1] = math.floor(item["Low"])
#        print("Updated the o value to:"+str(list2[-1]))
        numberofOBoxes += 1
    elif ( math.floor(item["High"]) >= list2[-1]+reversal*box_size):
        list1.append(math.floor(item["High"]))
 #       print("Updated the x value to:"+str(list1[-1]))
        current_trend = 'x'
        numberofXBoxes += 1
    return current_trend

#create the point and figure from scratch

def pointAndFigureCreate(box_size,current_trend, list1, list2, stockhilowdata, numberofXBoxes, numberofOBoxes):
    #iterate over each date for the stock and create x or o columns
    for index,row in stockhilowdata.iloc[0:].iterrows():
        if current_trend == 'x':
        #   print("The price is:"+str(row["High"]))
            current_trend = updateX(row, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes)
        elif current_trend=='o':
            current_trend = updateO(row, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes)
          #  print("The price is:"+str(row["Low"]))
    return current_trend

#check for a continous triple top pattern
def continous_triple_top(current_trend, list1):
    if(len(list1)<3):
        return -1
    ret_val = 0
    if current_trend == 'x':
        if((list1[-1]> list1[-2]) and (list1[-2] == list1[-3])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val
        
#check for a continous triple bottom pattern
def continous_triple_bottom(current_trend, list2):
    if(len(list2)<3):
        return -1
    ret_val = 0
    if current_trend == 'o':
        if((list2[-1] < list2[-2]) and (list2[-2] == list2[-3])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

#check for a continous double top pattern
def continous_double_top(current_trend, list1):
    if(len(list1)<2):
        return -1
    ret_val = 0
    if current_trend=='x':
        if(list1[-1]> list1[-2]):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

#check for a continous double bottom pattern
def continous_double_bottom(current_trend, list2):
    if(len(list2)<2):
        return -1
    ret_val = 0
    if current_trend == 'o':
        if(list2[-1]< list2[-2]):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def continous_quadruple_top(current_trend, list1):
    if(len(list1)<4):
        return -1
    ret_val = 0
    if current_trend == 'x' and len(list1)>3:
        if((list1[-1] > list1[-2]) and (list1[-2] == list1[-3] == list1[-4])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def continous_quadruple_bottom(current_trend, list2):
    if(len(list2)<4):
        return -1
    ret_val = 0
    if current_trend == 'o' and len(list2)>3:
        if((list2[-1] < list2[-2]) and (list2[-2] == list2[-3] == list2[-4])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

import itertools
def spread_triple_top(current_trend, list1):
    ret_val = -1
    if current_trend == 'x' and len(list1)>7:
        iterprev = 0
        resistance = 0
        notstt = True
        for iter in list1[-2:-7:1]:
            if iterprev == iter:
                resistance = iter
                notstt = False
            if resistance!=0 and iter > resistance:
                notstt = True
                break
            iterprev = iter
        if(notstt == False):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def spread_triple_bottom(current_trend, list2):
    ret_val = -1
    if current_trend == 'o' and len(list2)>7:
        iterprev = 0
        support = 0
        notstb = True
        for iter in list2[-2:-7:1]:
            if iterprev == iter:
                support = iter
                notstb = False
            if support!=0 and iter<resistance:
                notstb = True
                break
            iterprev = iter
        if(notstb == False):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def PnFMain(myData):
    #list1 stores the highest value of each x column by index
    #list2 stores the lowest value of each o column by index
    list1 = []
    list2 = []

    #current_trend stores the value of the most recent trend. we start off with x as default
    current_trend = 'x'

    numberofXBoxes = 1 # to get number of x boxes in all
    numberofOBoxes = 0 # to get number of o boxes in all

    #set box size according to price on day 1
    box_size = updateBoxSize(myData["High"].iloc[0])
    #append price on day 1 to list1 as we start with x
    list1.append(math.floor(myData["High"].iloc[0]))
    current_trend = pointAndFigureCreate(box_size, current_trend, list1, list2, myData, numberofXBoxes, numberofOBoxes)
    ret_val_triple_top = continous_triple_top(current_trend,list1)
    ret_val_triple_bottom = continous_triple_bottom(current_trend,list2)
    ret_val_double_top = continous_double_top(current_trend,list1)
    ret_val_double_bottom = continous_double_bottom(current_trend,list2)
    ret_val_quadruple_top = continous_quadruple_top(current_trend,list1)
    ret_val_quadruple_bottom = continous_quadruple_bottom(current_trend,list2)
    ret_val_spread_triple_top = spread_triple_top(current_trend,list1)
    ret_val_spread_triple_bottom = spread_triple_bottom(current_trend,list2)
    
    return ret_val_double_top

In [20]:
df_rated_now=pd.DataFrame()

In [203]:
for sector in sector_etf_list:
#     if sector_sma_rating[sector]==1 and sector_macd_rating[sector]==1 and sector_macd_slope[sector]==1:
        for i in range(0,len(sector_etf_to_names[sector])):
            time.sleep(15)
            myData=get_all_data(sector_etf_to_names[sector][i])
            time.sleep(15)
            if(not myData.empty):
                macd_rating,slope = MACD_rating(myData)
                df_rated_now= df_rated_now.append({'Stock Symbol':sector_etf_to_names[sector][i], 'SMA':SMA_rating(myData), 'MACD':macd_rating,'MACD Slope':slope, 'PnF':PnFMain(myData), 'RSI':daily_rsi_values(sector_etf_to_names[sector][i]) }, ignore_index=True)
            else:
                print('Data for stock not present in the API')
            print(df_rated_now)

Current: -0.18446111487931205
Previous day -0.2120822610439319
   MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0   5.0         1.0  0.0  1.0  4.0          AMG
Current: 0.9886067717861806
Previous day 1.0098603478732144
   MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0   5.0         1.0  0.0  1.0  4.0          AMG
1   1.0        -1.0  1.0  1.0  1.0          AFL
Current: 1.496584906397402
Previous day 1.4794985799040603
   MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0   5.0         1.0  0.0  1.0  4.0          AMG
1   1.0        -1.0  1.0  1.0  1.0          AFL
2   1.0         1.0  1.0  1.0  1.0          ALL
Current: 1.5744674347869676
Previous day 1.5981162632914163
   MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0   5.0         1.0  0.0  1.0  4.0          AMG
1   1.0        -1.0  1.0  1.0  1.0          AFL
2   1.0         1.0  1.0  1.0  1.0          ALL
3   2.0        -1.0  1.0  1.0  1.0          AXP
Current: 0.7677836946119143
Previous day 0.7133606893649542
   MACD  MACD Slope  PnF  

Current: -0.699423988561513
Previous day -0.709634928642366
    MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0    5.0         1.0  0.0  1.0  4.0          AMG
1    1.0        -1.0  1.0  1.0  1.0          AFL
2    1.0         1.0  1.0  1.0  1.0          ALL
3    2.0        -1.0  1.0  1.0  1.0          AXP
4    1.0         1.0  1.0  1.0  1.0          AIG
5    1.0         1.0 -1.0  1.0  1.0          AMP
6    2.0        -1.0  1.0  1.0  1.0          AON
7    1.0         1.0  1.0  1.0  1.0          AJG
8    1.0         1.0  1.0  1.0  1.0          AIZ
9    1.0         1.0 -1.0  1.0  1.0          BAC
10   1.0         1.0 -1.0  1.0  1.0          BBT
11   1.0        -1.0 -1.0 -1.0  1.0        BRK.B
12   1.0        -1.0 -1.0  1.0  1.0          HRB
13   5.0         1.0  0.0 -1.0  4.0          BHF
14   1.0         1.0  1.0  1.0  1.0          COF
15   1.0         1.0 -1.0  1.0  1.0         CBOE
16   5.0         1.0 -1.0  1.0  4.0         SCHW
Current: 0.6247476354842547
Previous day 0.661647498642253

Current: -0.38608251282256845
Previous day -0.38408357607591626
    MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0    5.0         1.0  0.0  1.0  4.0          AMG
1    1.0        -1.0  1.0  1.0  1.0          AFL
2    1.0         1.0  1.0  1.0  1.0          ALL
3    2.0        -1.0  1.0  1.0  1.0          AXP
4    1.0         1.0  1.0  1.0  1.0          AIG
5    1.0         1.0 -1.0  1.0  1.0          AMP
6    2.0        -1.0  1.0  1.0  1.0          AON
7    1.0         1.0  1.0  1.0  1.0          AJG
8    1.0         1.0  1.0  1.0  1.0          AIZ
9    1.0         1.0 -1.0  1.0  1.0          BAC
10   1.0         1.0 -1.0  1.0  1.0          BBT
11   1.0        -1.0 -1.0 -1.0  1.0        BRK.B
12   1.0        -1.0 -1.0  1.0  1.0          HRB
13   5.0         1.0  0.0 -1.0  4.0          BHF
14   1.0         1.0  1.0  1.0  1.0          COF
15   1.0         1.0 -1.0  1.0  1.0         CBOE
16   5.0         1.0 -1.0  1.0  4.0         SCHW
17   2.0        -1.0  0.0  1.0  1.0           CB
18   

Current: 0.13184793575315723
Previous day 0.1291651062187178
    MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0    5.0         1.0  0.0  1.0  4.0          AMG
1    1.0        -1.0  1.0  1.0  1.0          AFL
2    1.0         1.0  1.0  1.0  1.0          ALL
3    2.0        -1.0  1.0  1.0  1.0          AXP
4    1.0         1.0  1.0  1.0  1.0          AIG
5    1.0         1.0 -1.0  1.0  1.0          AMP
6    2.0        -1.0  1.0  1.0  1.0          AON
7    1.0         1.0  1.0  1.0  1.0          AJG
8    1.0         1.0  1.0  1.0  1.0          AIZ
9    1.0         1.0 -1.0  1.0  1.0          BAC
10   1.0         1.0 -1.0  1.0  1.0          BBT
11   1.0        -1.0 -1.0 -1.0  1.0        BRK.B
12   1.0        -1.0 -1.0  1.0  1.0          HRB
13   5.0         1.0  0.0 -1.0  4.0          BHF
14   1.0         1.0  1.0  1.0  1.0          COF
15   1.0         1.0 -1.0  1.0  1.0         CBOE
16   5.0         1.0 -1.0  1.0  4.0         SCHW
17   2.0        -1.0  0.0  1.0  1.0           CB
18   1.0

Current: -0.17236137835593368
Previous day -0.16333407424397706
    MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0    5.0         1.0  0.0  1.0  4.0          AMG
1    1.0        -1.0  1.0  1.0  1.0          AFL
2    1.0         1.0  1.0  1.0  1.0          ALL
3    2.0        -1.0  1.0  1.0  1.0          AXP
4    1.0         1.0  1.0  1.0  1.0          AIG
5    1.0         1.0 -1.0  1.0  1.0          AMP
6    2.0        -1.0  1.0  1.0  1.0          AON
7    1.0         1.0  1.0  1.0  1.0          AJG
8    1.0         1.0  1.0  1.0  1.0          AIZ
9    1.0         1.0 -1.0  1.0  1.0          BAC
10   1.0         1.0 -1.0  1.0  1.0          BBT
11   1.0        -1.0 -1.0 -1.0  1.0        BRK.B
12   1.0        -1.0 -1.0  1.0  1.0          HRB
13   5.0         1.0  0.0 -1.0  4.0          BHF
14   1.0         1.0  1.0  1.0  1.0          COF
15   1.0         1.0 -1.0  1.0  1.0         CBOE
16   5.0         1.0 -1.0  1.0  4.0         SCHW
17   2.0        -1.0  0.0  1.0  1.0           CB
18   

...

Current: -0.0831778568266941
Previous day -0.07151690041330028
    MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0    5.0         1.0  0.0  1.0  4.0          AMG
1    1.0        -1.0  1.0  1.0  1.0          AFL
2    1.0         1.0  1.0  1.0  1.0          ALL
3    2.0        -1.0  1.0  1.0  1.0          AXP
4    1.0         1.0  1.0  1.0  1.0          AIG
5    1.0         1.0 -1.0  1.0  1.0          AMP
6    2.0        -1.0  1.0  1.0  1.0          AON
7    1.0         1.0  1.0  1.0  1.0          AJG
8    1.0         1.0  1.0  1.0  1.0          AIZ
9    1.0         1.0 -1.0  1.0  1.0          BAC
10   1.0         1.0 -1.0  1.0  1.0          BBT
11   1.0        -1.0 -1.0 -1.0  1.0        BRK.B
12   1.0        -1.0 -1.0  1.0  1.0          HRB
13   5.0         1.0  0.0 -1.0  4.0          BHF
14   1.0         1.0  1.0  1.0  1.0          COF
15   1.0         1.0 -1.0  1.0  1.0         CBOE
16   5.0         1.0 -1.0  1.0  4.0         SCHW
17   2.0        -1.0  0.0  1.0  1.0           CB
18   1

Current: 1.023125803200216
Previous day 1.0004389424784677
    MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0    5.0         1.0  0.0  1.0  4.0          AMG
1    1.0        -1.0  1.0  1.0  1.0          AFL
2    1.0         1.0  1.0  1.0  1.0          ALL
3    2.0        -1.0  1.0  1.0  1.0          AXP
4    1.0         1.0  1.0  1.0  1.0          AIG
5    1.0         1.0 -1.0  1.0  1.0          AMP
6    2.0        -1.0  1.0  1.0  1.0          AON
7    1.0         1.0  1.0  1.0  1.0          AJG
8    1.0         1.0  1.0  1.0  1.0          AIZ
9    1.0         1.0 -1.0  1.0  1.0          BAC
10   1.0         1.0 -1.0  1.0  1.0          BBT
11   1.0        -1.0 -1.0 -1.0  1.0        BRK.B
12   1.0        -1.0 -1.0  1.0  1.0          HRB
13   5.0         1.0  0.0 -1.0  4.0          BHF
14   1.0         1.0  1.0  1.0  1.0          COF
15   1.0         1.0 -1.0  1.0  1.0         CBOE
16   5.0         1.0 -1.0  1.0  4.0         SCHW
17   2.0        -1.0  0.0  1.0  1.0           CB
18   1.0  

Current: -0.44337742185750795
Previous day -0.44944630080661696
    MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0    5.0         1.0  0.0  1.0  4.0          AMG
1    1.0        -1.0  1.0  1.0  1.0          AFL
2    1.0         1.0  1.0  1.0  1.0          ALL
3    2.0        -1.0  1.0  1.0  1.0          AXP
4    1.0         1.0  1.0  1.0  1.0          AIG
5    1.0         1.0 -1.0  1.0  1.0          AMP
6    2.0        -1.0  1.0  1.0  1.0          AON
7    1.0         1.0  1.0  1.0  1.0          AJG
8    1.0         1.0  1.0  1.0  1.0          AIZ
9    1.0         1.0 -1.0  1.0  1.0          BAC
10   1.0         1.0 -1.0  1.0  1.0          BBT
11   1.0        -1.0 -1.0 -1.0  1.0        BRK.B
12   1.0        -1.0 -1.0  1.0  1.0          HRB
13   5.0         1.0  0.0 -1.0  4.0          BHF
14   1.0         1.0  1.0  1.0  1.0          COF
15   1.0         1.0 -1.0  1.0  1.0         CBOE
16   5.0         1.0 -1.0  1.0  4.0         SCHW
17   2.0        -1.0  0.0  1.0  1.0           CB
18   

Current: 0.9574866515556835
Previous day 0.9915206445025717
    MACD  MACD Slope  PnF  RSI  SMA Stock Symbol
0    5.0         1.0  0.0  1.0  4.0          AMG
1    1.0        -1.0  1.0  1.0  1.0          AFL
2    1.0         1.0  1.0  1.0  1.0          ALL
3    2.0        -1.0  1.0  1.0  1.0          AXP
4    1.0         1.0  1.0  1.0  1.0          AIG
5    1.0         1.0 -1.0  1.0  1.0          AMP
6    2.0        -1.0  1.0  1.0  1.0          AON
7    1.0         1.0  1.0  1.0  1.0          AJG
8    1.0         1.0  1.0  1.0  1.0          AIZ
9    1.0         1.0 -1.0  1.0  1.0          BAC
10   1.0         1.0 -1.0  1.0  1.0          BBT
11   1.0        -1.0 -1.0 -1.0  1.0        BRK.B
12   1.0        -1.0 -1.0  1.0  1.0          HRB
13   5.0         1.0  0.0 -1.0  4.0          BHF
14   1.0         1.0  1.0  1.0  1.0          COF
15   1.0         1.0 -1.0  1.0  1.0         CBOE
16   5.0         1.0 -1.0  1.0  4.0         SCHW
17   2.0        -1.0  0.0  1.0  1.0           CB
18   1.0 

KeyboardInterrupt: 

In [204]:
def get_final_list():
    final_list = list()
    for index,row in df_rated_now.iloc[0:].iterrows():
        if row['MACD'] == 1 and row['MACD Slope'] == 1 and row['SMA'] == 1:
            final_list.append(row['Stock Symbol'])
    return final_list

In [ ]:
import requests
import json
import csv
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime  
from datetime import timedelta


# Following code makes relevant tokens necessary to conduct TD ameritrade activities on behalf of my account
# Note refresh token is valid for 90 days. It is used to generate auth token which is only valid for 30 mins
def get_auth_token():
    refresh_token = 'sXtUVLkSa8gbLhTjgVRPJDkx0J2khZOwk6uN+q0KlnrLoNuz6Fnm893VKp76zqml0GWZ73K1Oler9JJYMNFxziNe58G+SQ5zLgqLdA3EXVO3FXIPSR74glOqGddJ2hKH9tXL1oWft5WCE6ZUpq/zjPhsJSBPi8yGx2RnSABUc7FMqTeYOUcBztjjCUl4/jgk2F076hyrrDCSBX/yakGHA09jHcbQZOOl8vyrxu/UfrjEgfxNi4YuEMKz23pcHd2L77dJRBMTugYYnFzRpHMHdGukWedlgOO4/RgQgsZ3nAKEFrSYUS9XrugrREalD2Ufo+onBbkr+N1EP+BEyinufwOkrwsebgw/5FQKB9fJzbNw/79nXeS0ZG4OHlHA3shPP6JbJOUeh2Fwev/yuhrE6sGwfcdvftzn8vFTe52KGJrBe5RapEb+n+DxkJm100MQuG4LYrgoVi/JHHvlrueqQYnI53tYFD84wI4dnFH19sX584k0XZTqmBn4M2JDQqiO4SRIMAOVtkogDmlhOHMo1xDZVUG1aDRSdnoXe3EbnfnYI8h/PfIRyBUeVolmaUDfR5KUQ7etO7BMYlfC4w0hKhbcl1HayMZLi9lgtCPqW2cjoonhrywNt+TjlwG1zwxdfBL+T2YJmyb4jGYKe3r+/LfVV/Mo73zzodDajQtkCLjbtCcGPs8SrETi73eslhHuVxhUs73PBJBhwQAnwFXy1WsgNHANr3M733dxb/kJqlSkJTTN416SeuMuu69pM8j/T7AW1+Ly/d4OVjiIhpp5ZlU9U9YrSVdSQtD9A3Tt/VHI3jIHNn0yDMyweZlwfzzHKXDVEemNXPWXu3iGj/VZqujZ3RL0ZnpNHbQDf8PX+LjkY+0e7HteljkhgZlg4X0Q5F4YySluQCE=212FD3x19z9sWBHDJACbC00B75E'
    auth_params = {'grant_type':'refresh_token', 'refresh_token':refresh_token, 'client_id':'TESTAUTH122@AMER.OAUTHAP'}
    auth_api_url = 'https://api.tdameritrade.com/v1/oauth2/token'
    return "Bearer "+ requests.post(auth_api_url, data=auth_params).json()['access_token']


# Following code will get the option chain data using the authorized token
# Parameters: call_or_put: 'CALL'/'PUT'; symbol: string with stock symbol from passed in stock list
def get_raw_option_data(call_or_put, symbol):
    url = 'https://api.tdameritrade.com/v1/marketdata/chains'
    strike_count = '500000' #need to figure out how to get ALL or just use an arbitraliy huge num
    strategy = 'SINGLE'
    authorization_header = {'Authorization': get_auth_token()}
    pay = {'symbol':symbol,'strikeCount':strike_count, 'strategy':strategy}
    if call_or_put == "CALL":
        return [requests.get(url, params = pay, headers = authorization_header).json()['callExpDateMap'], requests.get(url, params = pay, headers = authorization_header).json()['underlyingPrice']]
    elif call_or_put == "PUT":
        return [requests.get(url, params = pay, headers = authorization_header).json()['putExpDateMap'], requests.get(url, params = pay, headers = authorization_header).json()['underlyingPrice']]



# Following code does the filtering of the options data based on our chosen parameters
def processing(symbol, params, call_or_put):
    data = get_raw_option_data(call_or_put, symbol)
    agg_data = {}
    date_keys = []
    for (k,v) in data[0].items():
        date_keys.append([k,list(v.keys())])
        agg_data[k] = v
        
    #If there is no data regarding some symbol in API then just return an empty dataframe
    if not date_keys:
        return pd.DataFrame()
        
    print(agg_data[date_keys[0][0]][date_keys[0][1][0]][0]['putCall']) #This is an example of how we can access data. Can make less complicated if necessary using a different data structure but this should be good enough performace-wise
    target_list = []
    stock_price = data[1]
    if call_or_put == "CALL":
        for i in range(0,len(date_keys)):
            for j in range(0, len(date_keys[i][1])):
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < params['days_min'] or agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] > params['days_max']: # Days to Expiration Requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] < params['bid_min']: # Min/Max Bid requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['totalVolume'] < params['min_vol']: # Min/Max Volume Requirement!
                    continue
                if (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price) / stock_price < params['%OTM_lim']: # Percent OTM Requirement!
                    continue
                if 1 - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'] < params['prob_OTM_lim']: # Prob OTM Requirement!
                    continue
                # construct a dictionary object characteristics we want and append to list
                target = {'symbol': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['symbol'],
                         'strike_price':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
                         'days_to_exp':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'],
                         'bid': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'],
                         'ask': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'],
                         'last': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['last'],
                         'prob_OTM': 1 - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'],
                         '% OTM': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price) / stock_price,
                         'cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'])/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'max_cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']+((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'prob_touch': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*2}
                target_list.append(target)
    elif call_or_put == "PUT":
        for i in range(0,len(date_keys)):
            for j in range(0, len(date_keys[i][1])):
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < params['days_min'] or agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] > params['days_max']: # Days to Expiration Requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] < params['bid_min']: # Min/Max Bid requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['totalVolume'] < params['min_vol']: # Min/Max Volume Requirement!
                    continue
                if (stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']) / stock_price < params['%OTM_lim']: # Percent OTM Requirement!
                    continue
                if 1 + agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'] < params['prob_OTM_lim']: # Prob OTM Requirement!      + or -???
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']) < params['cov_ret']: # Covered return Requirement! 
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*-2 < params['prob_touch']: # Probability of touching parameter      + or - ???
                    continue
                    
#Commented code checks for 24% annual return. If required in future
#                 value = (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid']*365*100)/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']*agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'])
#                 if value<15:
#                     continue
    
                # Construct a dictionary object characteristics we want and append to list
                target = {'symbol': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['symbol'],
                          'strike_price':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
                          'days_to_exp':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'],
                         'bid': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'],
                         'ask': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'],
                         'last': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['last'],
                         'prob_OTM': 1 + agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'], # + or - ???
                         '% OTM': stock_price/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
                         'cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'max_cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']+((stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'])/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'prob_touch': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*2} # + or - ???
                target_list.append(target)
    df = pd.DataFrame(target_list)
    if(df.empty):
        return df
    df = df.set_index('symbol', drop = False)
    return df



# Following function returns price data for a stock which can be used to construct a P&F chart
def return_candle_data(symbol):
    candle_url = 'https://api.tdameritrade.com/v1/marketdata/'+symbol+'/pricehistory'
    candle_header = {'Authorization': get_auth_token()}
    # Vary the properties below depending on how often you want the data
    # Ideally, we want to set up a web socket to continuously stream this data?
    period_type = 'day'
    period = 2
    freq_type = 'minute'
    freq = 1
    candle_payload = {'periodType': period_type, 'period': period, 'frequencyType': freq_type, 'frequency': freq}
    raw_candle_data = requests.get(candle_url, params = candle_payload, headers = candle_header).json()
    df = pd.DataFrame(raw_candle_data['candles'])
    return df


# Get the relevant options after proving stock symbol and parameter list for filtering. 
def main():
#     stock_list = get_final_list() # Need to populate this array with stock list we derive from technical indicators
    stock_list = lis1
    print(stock_list)
    default_call_params = {'days_min': 1, 'days_max': 30, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.15, 'prob_OTM_lim': 0.15}
    charlie_call_params = {'days_min': 1, 'days_max': 30, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.15, 'prob_OTM_lim': 0.15}
    default_put_params = {'days_min': 1, 'days_max': 30, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.15, 'prob_OTM_lim': 0.15, 'cov_ret': 0.01, 'prob_touch': 0.01}
    charlie_put_params = {'days_min': 1, 'days_max': 60, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.15, 'prob_OTM_lim': 0.15, 'cov_ret': 0.01, 'prob_touch': 0.01}
    safe_put_parameters = {'days_min': 1, 'days_max': 60, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.05, 'prob_OTM_lim': 0.05, 'cov_ret': 0.01, 'prob_touch': 0.01}
    mod_put_parameters = {'days_min': 1, 'days_max': 60, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.10, 'prob_OTM_lim': 0.10, 'cov_ret': 0.01, 'prob_touch': 0.01}
    risky_put_parameters = {'days_min': 1, 'days_max': 60, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.15, 'prob_OTM_lim': 0.15, 'cov_ret': 0.01, 'prob_touch': 0.01}
    df_final = pd.DataFrame()
    d_final = dict()
    for item in stock_list:
        print(item)
        df = processing(item, safe_put_parameters, "PUT")
        if(not df.empty):
            df_top_ten = df.sort_values(by=['bid'], ascending=False)
            print("The maximum ordering for stock", item)
            print(df_top_ten[0:len(df_top_ten)]['bid'])
            df_top_ten = df_top_ten.head(10)
            print("Not empty")
            df.to_csv(item+".csv")
#             d_final["Symbol"] = item
#             d_final["Contracts"] = df.to_json(orient='records')[1:-1].replace('},{', '} {')
#     ans = pd.DataFrame.from_dict(d_final.items())
#     print(d_final["Contracts"])    
    
if __name__ == "__main__":
    main()


1        AOS
2        ABT
3       ABBV
4        ACN
5       ATVI
6        AYI
7       ADBE
8        AAP
9        AMD
10       AES
11       AET
12       AMG
13       AFL
14         A
15       APD
16      AKAM
17       ALK
18       ALB
19       ARE
20      ALXN
21      ALGN
22      ALLE
23       AGN
24       ADS
25       LNT
26       ALL
27     GOOGL
28      GOOG
29        MO
30      AMZN
       ...  
475     VRTX
476     VIAB
477        V
478      VNO
479      VMC
480      WMT
481      WBA
482       WM
483      WAT
484      WEC
485      WFC
486     WELL
487      WDC
488       WU
489      WRK
490       WY
491      WHR
492      WMB
493     WLTW
494      WYN
495     WYNN
496      XEL
497      XRX
498     XLNX
499       XL
500      XYL
501      YUM
502      ZBH
503     ZION
504      ZTS
Name: Symbol, Length: 504, dtype: object
AOS
PUT
ABT
PUT
The maximum ordering for stock ABT
symbol
ABT_081619P80      0.57
ABT_082319P78.5    0.49
ABT_080919P80      0.47
Name: bid, dtype: float64
Not empty


PUT
AAL
PUT
The maximum ordering for stock AAL
symbol
AAL_082319P31      0.82
AAL_081619P31      0.74
AAL_082319P30.5    0.65
AAL_081619P30      0.52
AAL_080919P30.5    0.50
AAL_080219P31      0.40
AAL_072619P31      0.38
AAL_081619P29      0.32
Name: bid, dtype: float64
Not empty
AEP
PUT
The maximum ordering for stock AEP
symbol
AEP_081619P85    0.35
Name: bid, dtype: float64
Not empty
AXP
PUT
The maximum ordering for stock AXP
symbol
AXP_082319P120    1.09
AXP_080919P118    0.56
AXP_072619P120    0.54
AXP_072619P119    0.43
AXP_071919P120    0.40
AXP_080219P117    0.38
AXP_071919P119    0.32
AXP_080219P115    0.28
Name: bid, dtype: float64
Not empty
AIG
PUT
The maximum ordering for stock AIG
symbol
AIG_081619P52.5    0.55
Name: bid, dtype: float64
Not empty
AMT
PUT
The maximum ordering for stock AMT
symbol
AMT_081619P200      1.35
AMT_081619P195      0.80
AMT_080219P200      0.75
AMT_080219P197.5    0.55
AMT_081619P190      0.45
Name: bid, dtype: float64
Not empty
AWK
PUT
AMP
PUT
The

PUT
CHRW
PUT
The maximum ordering for stock CHRW
symbol
CHRW_081619P77.5    0.95
CHRW_081619P75      0.60
Name: bid, dtype: float64
Not empty
CA
COG
PUT
The maximum ordering for stock COG
symbol
COG_081619P23    0.45
COG_082319P22    0.25
Name: bid, dtype: float64
Not empty
CDNS
PUT
The maximum ordering for stock CDNS
symbol
CDNS_081619P70    1.00
CDNS_081619P65    0.25
Name: bid, dtype: float64
Not empty
CPB
PUT
The maximum ordering for stock CPB
symbol
CPB_082319P38.5    0.3
Name: bid, dtype: float64
Not empty
COF
PUT
The maximum ordering for stock COF
symbol
COF_081619P85      1.05
COF_080919P83.5    0.67
COF_081619P82.5    0.62
COF_080919P82      0.48
COF_071919P85      0.41
COF_081619P80      0.40
COF_071919P84      0.29
Name: bid, dtype: float64
Not empty
CAH
PUT
The maximum ordering for stock CAH
symbol
CAH_081619P45      0.9
CAH_080219P44.5    0.3
Name: bid, dtype: float64
Not empty
KMX
PUT
The maximum ordering for stock KMX
symbol
KMX_081619P82.5    0.7
Name: bid, dtype: float

PUT
The maximum ordering for stock DAL
symbol
DAL_082319P56      0.84
DAL_082319P55      0.64
DAL_081619P55      0.49
DAL_082319P54      0.48
DAL_080919P56      0.47
DAL_080219P56      0.40
DAL_080919P55      0.34
DAL_071919P56.5    0.25
DAL_072619P56      0.25
Name: bid, dtype: float64
Not empty
XRAY
PUT
The maximum ordering for stock XRAY
symbol
XRAY_071919P57.5    0.7
Name: bid, dtype: float64
Not empty
DVN
PUT
The maximum ordering for stock DVN
symbol
DVN_082319P26.5    0.62
DVN_080919P26.5    0.44
DVN_080919P26      0.34
Name: bid, dtype: float64
Not empty
DLR
PUT
DFS
PUT
DISCA
PUT
The maximum ordering for stock DISCA
symbol
DISCA_081619P30    0.6
Name: bid, dtype: float64
Not empty
DISCK
PUT
DISH
PUT
The maximum ordering for stock DISH
symbol
DISH_081619P37.5    1.10
DISH_080919P38      0.65
DISH_080919P37      0.35
DISH_080919P37.5    0.30
DISH_072619P37.5    0.25
DISH_080919P36.5    0.25
Name: bid, dtype: float64
Not empty
DG
PUT
The maximum ordering for stock DG
symbol
DG_0823

PUT
The maximum ordering for stock HCA
symbol
HCA_081619P130    1.9
HCA_081619P120    0.6
HCA_081619P115    0.3
Name: bid, dtype: float64
Not empty
HCP
PUT
HP
PUT
The maximum ordering for stock HP
symbol
HP_081619P45    0.35
Name: bid, dtype: float64
Not empty
HSIC
PUT
The maximum ordering for stock HSIC
symbol
HSIC_071919P70      1.25
HSIC_071919P67.5    0.25
Name: bid, dtype: float64
Not empty
HES
PUT
The maximum ordering for stock HES
symbol
HES_082319P60      1.53
HES_081619P60      1.36
HES_082319P58      1.04
HES_081619P57.5    0.81
HES_080919P57.5    0.62
HES_081619P55      0.45
Name: bid, dtype: float64
Not empty
HPE
PUT
HLT
PUT
HOLX
PUT
HD
PUT
The maximum ordering for stock HD
symbol
HD_081619P200      1.06
HD_080919P200      0.80
HD_081619P195      0.60
HD_080919P197.5    0.58
HD_080219P200      0.55
HD_080219P197.5    0.38
HD_081619P190      0.36
HD_072619P200      0.31
HD_080219P195      0.27
HD_080919P190      0.25
Name: bid, dtype: float64
Not empty
HON
PUT
The maximum or